In [12]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
from matplotlib.gridspec import GridSpec
from scipy.ndimage import gaussian_filter
import subjects
import time
from ModulesPath.utils import position_util

# sessions = subjects.Sd().ratRday2
sessions = subjects.Nsd().ratRday1
# sessions = subjects.Sd().utkuAG_day2
# sessions = subjects.Of().ratNday4


## Generate _basics.npy files

In [7]:
for sub, sess in enumerate(sessions):
    # openephys_settingspath = (
    #     "/data/Clustering/SleepDeprivation/RatS/Day3SD/openEphysSettings/settings.xml"
    # )
    # sess.recinfo.generate_xml(settingsPath=openephys_settingspath)
    # sess.recinfo.makerecinfo(nShanks=[6, 8], skulleeg=[51], motion=[192, 193, 194])
    sess.recinfo.makerecinfo(nShanks=8)
    # sess.recinfo.makerecinfo(nShanks=8)
    print(sess.recinfo)

_basics.npy created for RatRDay1NSD
Name: RatR 
Channels: 128
Sampling Freq: 30000
lfp Srate (downsampled): 1250
# bad channels: 49
motion channels: []
emg channels: []
skull eeg: []


## Generate probemap and .prb for spyking circus

In [ ]:
%matplotlib widget
for sub, sess in enumerate(sessions):

    # sess.trange = np.array([])
    # sess.recinfo.probemap.create(xypitch=(16, 15))
    # sess.recinfo.probemap.plot()
    sess.recinfo.probemap.for_spyking_circus(rmv_badchans=True, shanksCombine=True)


## artifacts file gen

In [13]:
%matplotlib widget
for sub, sess in enumerate(sessions):
    # maze = sess.paradigm.maze1
    # lfp = sess.recinfo.geteeg(chans=64)
    # plt.plot(np.linspace(0, len(lfp) / (1250 * 60), len(lfp)), lfp)
    # zsc = sess.artifact.detect(chans=[29, 60], thresh=4)
    # sess.artifact.plot()
    sess.artifact.to_spyking_circus()


In [11]:
sess.artifact.epochs

,start,stop,label,duration
0,8450.2480,8450.2592,,0.0112
1,9549.3200,9549.8528,,0.5328
2,11911.7360,11916.6856,,4.9496
3,24022.8144,24494.7560,,471.9416
4,42963.7136,42963.8872,,0.1736
5,43170.1928,43171.3064,,1.1136
6,43910.1056,43910.1544,,0.0488
7,44159.0040,44163.0328,,4.0288
8,44276.1768,44276.2552,,0.0784
9,44287.2768,44300.6312,,13.3544


## Generate position files

In [ ]:
# region
for sub, sess in enumerate(sessions):
    sess.position.getPosition(method="from_metadata", scale=1.0)
    # sess.position.export2Neuroscope()

# endregion


## View spectrogram

In [ ]:
%matplotlib widget
for sub,sess in enumerate(sessions):
    sess.viewdata.specgram()

## create epochs

In [ ]:
for sub, sess in enumerate(sessions):

    a = np.array([13498145, 20798529, 9776812, 13909473])
    duration = a / (1250)
    start = np.append(0, np.cumsum(duration)[:-1])
    stop = start + duration

    epochs = pd.DataFrame(
        {
            "start": start,
            "stop": stop,
            "label": ["pre", "sd", "maze", "post"],
        }
    )
    # epochs = {
    #     "pre": [0, 11066],
    #     "maze1": [11070, 13970],
    #     "maze2": [20756, 24004],
    #     # "sd": [6033, 14818 + 5 * 3600],
    #     "post1": [13972, 20754],
    #     "post2": [24006, 42305],
    #     # "maze2": [36902, 40233],
    # }
    sess.paradigm.add_epochs(epochs)
    # sess.epochs.getfromCSV()


## Create tracks

In [ ]:
# region
for sub, sess in enumerate(sessions):
    # sess.trange = np.array([])
    # sess.tracks.create(epoch_names=["maze1", "maze2"])
    sess.tracks.linearize_position(track_names=["maze2"], sample_sec=5)
    sess.tracks.estimate_run_laps("maze1", speedthresh=(18, 20))
    # sess.epochs.getfromCSV()

# endregion


## Gen spikes or instantenous firing rate

In [ ]:
for sub, sess in enumerate(sessions):
    # sess.recinfo.sampfreq = 30000
    maze = sess.paradigm.maze1
    sess.neurons.from_Phy(
        folder="/home/bapung/Documents/ClusteringHub/spykcirc/RatR/Day2SD/Allsession/RatR_Day2SD_2021-05-23_09-07-18-1.GUI",
        fileformat="same_folder",
    )
    # sess.spikes.gen_instfiring()

## label cells

In [ ]:
# region
for sub, sess in enumerate(sessions):
    # sess.recinfo.sampfreq = 30000
    sess.spikes.label_celltype()


# endregion

## Generate bestThetachan

In [ ]:
# region

for sub, sess in enumerate(sessions):
    sess.trange = np.array([])
    sess.theta.detectBestChan()
# endregion


## Detect pbe

In [ ]:
# region
for sub, sess in enumerate(sessions):
    sess.pbe.detect()

# endregion


## Detect ripples

In [ ]:
for sub, sess in enumerate(sessions):
    # sess.ripple.channels()
    sess.ripple.detect()
    # sess.ripple.epochs

In [ ]:
sess.recinfo.lfpSrate

## Detect sleep states

In [ ]:
for sub, sess in enumerate(sessions):
    sess.brainstates.detect()
    # sess.epochs.maze


In [ ]:
# sess.brainstates.epochs
sess.brainstates.plot_hypnogram()